In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pwd

'/content'

In [2]:
cd /content/drive/MyDrive/GomokuAI

/content/drive/MyDrive/GomokuAI


# Git

In [ ]:
!git pull

remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (1/1), done.
remote: Total 3 (delta 2), reused 3 (delta 2), pack-reused 0
Unpacking objects: 100% (3/3), 387 bytes | 0 bytes/s, done.
From https://github.com/onetwothr1/GomokuAI
   6f6a2e0..ea27938  main       -> origin/main
Updating 6f6a2e0..ea27938
Fast-forward
 utils.py | 8 ++++++++
 1 file changed, 8 insertions(+)


In [3]:
!git add .

^C


In [4]:
!git config --global user.email kimjh1101@snu.ac.kr 
!git config --global user.name onetwotr1

In [5]:
!git commit -m "alphazero 2250"

^C


In [6]:
!git push origin main

^C


# Train

In [ ]:
import importlib
import train, utils
importlib.reload(train)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

alphazero 0 -> 750번 자체 대국 12시간

alphazero 750 -> 500번 자체 대국 8시간

alphazero 1250 -> 500번 자체 대국 20시간 (rollout 400)

alphazero 1750

In [ ]:
!python train.py -m "models/alphazero 2000.pt" -save "models/l2 0.001" -d "experience/alphazero 2000 self-play 250x8.pickle" -n 1000 -lr 0.001 --lr-decay 0.95 --early-stop 10

num data: 43168
num mini-batch: 43
--------------------------------
Epoch 1
43it [00:08,  4.89it/s]
average loss 1.5032 (policy 1.0805,  value 0.4227)
--------------------------------
Epoch 2
43it [00:01, 32.83it/s]
average loss 1.3430 (policy 0.9885,  value 0.3544)
--------------------------------
Epoch 3
43it [00:01, 31.66it/s]
average loss 1.2941 (policy 0.9599,  value 0.3342)
--------------------------------
Epoch 4
43it [00:01, 31.52it/s]
average loss 1.2428 (policy 0.9417,  value 0.3010)
--------------------------------
Epoch 5
43it [00:01, 31.62it/s]
average loss 1.2080 (policy 0.9302,  value 0.2778)
--------------------------------
Epoch 6
43it [00:01, 31.26it/s]
average loss 1.1814 (policy 0.9200,  value 0.2614)
--------------------------------
Epoch 7
43it [00:01, 26.61it/s]
average loss 1.1576 (policy 0.9082,  value 0.2493)
--------------------------------
Epoch 8
43it [00:01, 26.26it/s]
average loss 1.1345 (policy 0.9010,  value 0.2335)
--------------------------------
Epoc

alphazero 0 80.pt를 선택! -> alphazero 750
750 vs 0 => 9:1

c=1.0 vs c=2.0 => 12:2 (p-value 0.012)

c=1.0 vs c=1.5 => 10:0

weight2 epoch 109 낙찰 -> alphazero 1250.pt


alphazero 1250 vs alphazero 750 => 20 : 0

alphazero 1250
rollout 400 vs 100 => 10:0

1750 : 1250 => 10:0  (아래의 1750-1) 을 사용)

alphazero 1750 후보

1) non weighted loss, no l2 constraint. epoch 60   (overfitting 되기 전이라 판단한 지점에서 끊음)

2) non weighted loss, l2 constraint (0.001) epoch 102  (맨 끝 epoch)

왠진 모르겠는데 l2 사용한 게 조금 더 빠르게 loss 떨어짐. policy, value 모두.



후보 1), 2) 비교 결과 2)가 3:17로 압승. value function overfitting이 우려됐으나 조금도 우려하지 않아도 되는 듯함

alphazero 2000 100 vs alphazero 2000 => 9 : 1
p-value 0.021484

# 성능 비교

In [ ]:
import importlib
import compare_performance, bot_v_bot, human_v_bot, utils
importlib.reload(compare_performance)
importlib.reload(bot_v_bot)
importlib.reload(human_v_bot)
importlib.reload(utils)

<module 'utils' from '/content/drive/MyDrive/GomokuAI/utils.py'>

In [ ]:
import importlib
import agent.alphazero_agent
importlib.reload(agent.alphazero_agent)

<module 'agent.alphazero_agent' from '/content/drive/MyDrive/GomokuAI/agent/alphazero_agent.py'>

In [ ]:
!python bot_v_bot.py

In [ ]:
!python compare_performance.py -m1 "models/l2 0.001/alphazero 2000 100.pt" -m2 "models/alphazero 2000.pt" -n 10 --verbose 4 --use-model-name

Streaming output truncated to the last 5000 lines.
H1     visit   2  prior 0.038  value -0.74
B2     visit   2  prior 0.002  value -0.38
C2     visit   2  prior 0.004  value -0.67
E2     visit   2  prior 0.015  value -0.79
H2     visit   2  prior 0.017  value -0.71
----------------------------
Player.black (alphazero 2000 100) C6
 8                   
 7                   
 6     ●  ●        
 5       ●○○      
 4     ○  ●        
 3           ●      
 2             ○    
 1                   
 0                   
   A B C D E F G H J
average depth: 2.30, max depth: 6
B7     visit 159  prior 0.071  value -0.15
D6     visit   7  prior 0.072  value 0.09
F7     visit   7  prior 0.029  value -0.78
G5     visit   6  prior 0.036  value -0.69
D0     visit   5  prior 0.008  value -0.31
C3     visit   5  prior 0.013  value -0.68
G3     visit   5  prior 0.014  value -0.80
F6     visit   5  prior 0.015  value -0.78
G7     visit   5  prior 0.014  value -0.70
H7     visit   5  prior 0.005  value -

In [ ]:
!python compare_performance.py -m1 "models/l2 0.001/alphazero 2000 100.pt" -m2 "models/alphazero 2000.pt" -n 10 --verbose 4 --use-model-name

# self play

In [ ]:
import importlib
import bot_v_bot, utils, agent.alphazero_agent, compare_performance, self_play
# import net.alphazero_net
# importlib.reload(bot_v_bot)
# importlib.reload(compare_performance)
importlib.reload(utils)
importlib.reload(self_play)
# importlib.reload(agent.alphazero_agent)
# importlib.reload(net.alphazero_net)

<module 'self_play' from '/content/drive/MyDrive/GomokuAI/self_play.py'>

In [ ]:
for i in range(28,75):
  !python self_play.py -n 5 -m "models/alphazero 2250.pt" --file-num {i} -rollout 400 -c 2 --noise-intensity 0.3 --alpha 0.2 --reward-decay 0.92 --verbose 4

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
H2     visit  11  prior 0.026  value 0.93
----------------------------
Player.white C3
 8                   
 7                   
 6     ●○●●      
 5       ○●●      
 4     ○●○○      
 3     ○    ●      
 2         ○        
 1                   
 0                   
   A B C D E F G H J
average depth: 2.15, max depth: 4
G4     visit  98  prior 0.053  value -0.60
C5     visit  84  prior 0.114  value -0.30
H5     visit  76  prior 0.025  value -0.41
E3     visit  56  prior 0.079  value 0.03
G7     visit  29  prior 0.129  value -0.48
D7     visit  12  prior 0.043  value -0.70
G5     visit  11  prior 0.014  value -0.65
F7     visit   6  prior 0.010  value -0.78
F1     visit   3  prior 0.004  value -0.28
C7     visit   3  prior 0.032  value -0.18
----------------------------
Player.black G4
 8                   
 7                   
 6     ●○●●      
 5       ○●●      
 4     ○●○○●    
 3     ○    ●      
 2         ○        
 1                   
 0 

# 데이터 합치기

In [ ]:
import os
_list = os.listdir('experience')

In [ ]:
from pprint import pprint
files = [i for i in _list if '2000' in i and '.pickle' in i]
print(len(files))
pprint(files)

In [ ]:
import importlib
import experience
importlib.reload(experience)

<module 'experience' from '/content/drive/MyDrive/GomokuAI/experience.py'>

In [ ]:
from experience import *

combine_saved_experiences(['experience/' + f for f in files], "experience/alphazero 2000 self-play 250.pickle")

experience/alphazero 2000 self-play 5.pickle


/content/drive/MyDrive/GomokuAI/experience.py:105: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  self.mcts_probs = torch.tensor(self.mcts_probs)
/content/drive/MyDrive/GomokuAI/experience.py:28: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  rewards_list.append(torch.tensor(collector.rewards))


113
experience/alphazero 2000 self-play 5 1.pickle
108
experience/alphazero 2000 self-play 5 2.pickle
97
experience/alphazero 2000 self-play 5 3.pickle
133
experience/alphazero 2000 self-play 5 4.pickle
103
experience/alphazero 2000 self-play 5 5.pickle
117
experience/alphazero 2000 self-play 5 6.pickle
110
experience/alphazero 2000 self-play 5 7.pickle
106
experience/alphazero 2000 self-play 5 8.pickle
106
experience/alphazero 2000 self-play 5 9.pickle
93
experience/alphazero 2000 self-play 5 10.pickle
116
experience/alphazero 2000 self-play 5 11.pickle
118
experience/alphazero 2000 self-play 5 12.pickle
102
experience/alphazero 2000 self-play 5 13.pickle
112
experience/alphazero 2000 self-play 5 14.pickle
84
experience/alphazero 2000 self-play 5 15.pickle
115
experience/alphazero 2000 self-play 5 16.pickle
84
experience/alphazero 2000 self-play 5 17.pickle
81
experience/alphazero 2000 self-play 5 18.pickle
108
experience/alphazero 2000 self-play 5 19.pickle
93
experience/alphazero 20

In [ ]:
def seconds_to_hms(seconds):
    hours = seconds // 3600
    remaining_seconds = seconds % 3600
    minutes = remaining_seconds // 60
    remaining_seconds = remaining_seconds % 60

    return hours, minutes, remaining_seconds

In [ ]:
txt_files = [i for i in _list if '2000' in i and '.pickle' in i]
for f in txt_files:
    'experience/' +f

In [ ]:
from experience import *

combined = ExperienceCollector()
combined.load_experience("experience/alphazero 2000 self-play 250.pickle")
augmented = augmentation(combined, 9)
print(len(augmented))
print(len(augmented.mcts_probs))
print(len(augmented.rewards))

43168
43168
43168


In [ ]:
augmented.save_experience("experience/alphazero 2000 self-play 250x8.pickle")

# Play

In [ ]:
import importlib
import human_v_bot
importlib.reload(human_v_bot)

<module 'human_v_bot' from '/content/drive/MyDrive/GomokuAI/human_v_bot.py'>

In [ ]:
!python human_v_bot.py